In [ ]:
import numpy as np
from scipy.optimize import differential_evolution
import math

# Constants
theta = math.radians(7)  # PRBM displacement angle in radians
SF = 1.2  # Safety factor for static loading
P_target = 90  # Target force (N)

materials = {
    "4130 Q&T@400 Steel": {"E": 207e9, "sigma_ut": 1627e6, "sigma_y": 1462e6, "S_f_prime": 830e6},
    "7075-T6 Aluminum": {"E": 71.7e9, "sigma_ut": 572e6, "sigma_y": 503e6, "S_f_prime": 290e6},
    "6061-T6 Aluminum": {"E": 68.9e9, "sigma_ut": 310e6, "sigma_y": 276e6, "S_f_prime": 96.5e6},
    "Ti-6Al-4V Titanium": {"E": 113.8e9, "sigma_ut": 950e6, "sigma_y": 880e6, "S_f_prime": 510e6},
}

# Correction factors for fatigue strength
c_surf = 0.95
c_size = 1
c_load = 1
c_reliab = 0.7
c_misc = 1

# Constraints
def constraints_static(x, material):
    L, w, t = x
    E = material["E"]
    sigma_y = material["sigma_y"]

    phi = theta + np.pi / 2
    n = -1 / np.tan(phi)
    gamma = 0.852144 - 0.0182867 * n
    I = (w * t**3) / 12
    K_theta = (
        2.654855
        - 0.0509896 * n
        + 0.0126749 * (n**2)
        - 0.00142039 * (n**3)
        + 0.0000584525 * (n**4)
    )
    K = gamma * K_theta * E * I / L
    P = K * theta / (gamma * L * (np.cos(theta) + n * np.sin(theta)))

    a = L * (1 - gamma * (1 - np.cos(theta)))
    b = gamma * L * np.sin(theta)
    c = t / 2
    sigma_max = (P * (a + n * b) * c / I) + (n * P / (w * t))
    return sigma_y - (sigma_max * SF)

def constraints_fatigue(x, material):
    L, w, t = x
    E = material["E"]
    sigma_ut = material["sigma_ut"]
    S_f_prime = material["S_f_prime"]

    phi = theta + np.pi / 2
    n = -1 / np.tan(phi)
    gamma = 0.852144 - 0.0182867 * n
    I = (w * t**3) / 12
    K_theta = (
        2.654855
        - 0.0509896 * n
        + 0.0126749 * (n**2)
        - 0.00142039 * (n**3)
        + 0.0000584525 * (n**4)
    )
    K = gamma * K_theta * E * I / L
    P = K * theta / (gamma * L * (np.cos(theta) + n * np.sin(theta)))

    a = L * (1 - gamma * (1 - np.cos(theta)))
    b = gamma * L * np.sin(theta)
    c = t / 2
    sigma_max = (P * (a + n * b) * c / I) + (n * P / (w * t))
    S_f = c_surf * c_size * c_load * c_reliab * c_misc * S_f_prime
    sigma_alt = (sigma_max - 0) / 2
    sigma_mean = (sigma_max + 0) / 2
    SF_fatigue = 1 / (sigma_alt / S_f + sigma_mean / sigma_ut)
    return SF_fatigue - SF

def constraints_force(x, material):
    L, w, t = x
    E = material["E"]
    phi = theta + np.pi / 2
    n = -1 / np.tan(phi)
    gamma = 0.852144 - 0.0182867 * n
    I = (w * t**3) / 12
    K_theta = (
        2.654855
        - 0.0509896 * n
        + 0.0126749 * (n**2)
        - 0.00142039 * (n**3)
        + 0.0000584525 * (n**4)
    )
    K = gamma * K_theta * E * I / L
    P = K * theta / (gamma * L * (np.cos(theta) + n * np.sin(theta)))
    return P - P_target

def fitness(x, material):
    L, w, t = x
    penalty = 0

    # Direct constraint: force should be exactly equal to the target force
    penalty += max(0, abs(constraints_force(x, material)))
    penalty += max(0, -constraints_static(x, material))
    penalty += max(0, -constraints_fatigue(x, material))

    # Penalty for bounds
    penalty += 1e6 * max(0, w - 0.1)  # Width upper bound
    penalty += 1e6 * max(0, 0.01 - w)  # Width lower bound
    penalty += 1e6 * max(0, t - 0.01)  # Thickness upper bound
    penalty += 1e6 * max(0, 0.0008 - t)  # Thickness lower bound
    penalty += 1e6 * max(0, L - 0.5)  # Length upper bound

    return L + penalty

# Bounds for variables
bounds = [(0.01, 0.5), (0.01, 0.1), (0.0008, 0.01)]  # (L, w, t)

for material_name, material_props in materials.items():
    result = differential_evolution(
        lambda x: fitness(x, material_props), bounds, strategy='best1bin', mutation=(0.5, 1), recombination=0.7, maxiter=1000, popsize=50, tol=1e-6
    )
    print(f"Material: {material_name}")
    print(f"Optimal Length (L): {result.x[0]:.4f} m")
    print(f"Width (w): {result.x[1]:.4f} m")
    print(f"Thickness (t): {result.x[2]:.4f} m")

Material: 4130 Q&T@400 Steel
Optimal Length (L): 0.0397 m
Width (w): 0.0500 m
Thickness (t): 0.0008 m
Material: 7075-T6 Aluminum
Optimal Length (L): 0.0669 m
Width (w): 0.0787 m
Thickness (t): 0.0014 m
Material: 6061-T6 Aluminum
Optimal Length (L): 0.2588 m
Width (w): 0.1000 m
Thickness (t): 0.0032 m
Material: Ti-6Al-4V Titanium
Optimal Length (L): 0.0384 m
Width (w): 0.0663 m
Thickness (t): 0.0009 m
